# Escenario Electoral

* El candidato A ganó el estado por un promedio de 4 puntos porcentuales, sin embargo, perdió el Distrito 29 ante el candidato B por un 22 %, lo que lo convierte en un valor atípico significativo.
* Los resultados finales para el distrito son 39% Candidato A, 61% Candidato B.
* Tiene la tarea de investigar esto para determinar si vale la pena realizar un seguimiento exhaustivo.
* Se convoca a un centenar de miembros del público en general que se reportaron como votando. 48 de los 100 reportaron haber votado por el Candidato A.
* ¿Qué reportarías?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

Pero, ¿qué pregunta estamos realmente queriendo responder? Quizás este sea uno bueno:

*¿Cuál es la probabilidad de que más del Candidato A obtuviera más votos de los informados?*

O para decirlo de otra manera:
* Hipótesis Nula - 39% de los votos fueron para el Candidato A y 61% para el Candidato B
* Hipótesis Alternativa - >39% de los votos fueron para el Candidato A y <61% para el Candidato B

In [ ]:
sigma = np.sqrt(0.39 * 0.61 / 100)
reported = 0.39
sample = 0.48

xs = np.linspace(0, 1, 500)
ps = norm.pdf(xs, sample, sigma)
plt.plot(xs, ps, label="Probabilidad muestral subyacente")
plt.axvline(reported, ls="--", label="Proporcion Reportado")
plt.fill_between(xs, ps, 0, alpha=0.2, where=xs>=reported, label="prob")
plt.legend();

Todo lo que nos dice la trama anterior es que las cosas podrían parecer sospechosas. Pero echemos un vistazo a la CDF para obtener detalles y luego cuantifiquemos la probabilidad.

In [ ]:
prob_more = norm.sf(reported, sample, sigma)
print(f"Hay un  {100 * prob_more:.1f}% posibilidad de que el Candidato A hubiera recibido más votos")

Si queremos expresar esto en términos de puntajes z para una prueba de una cola y verificar si tenemos un valor p de <0.05

In [ ]:
z_score = (sample - reported) / sigma
z_score_needed_for_significance = norm.ppf(0.95)
print(f"{z_score:.3f} es mayor que {z_score_needed_for_significance:.3f}")
print(f"Tiene p-value {norm.sf(sample, reported, sigma):.3f}")

Entonces, ¿qué significa esto? Probablemente deberíamos decirles a nuestros supervisores que este es un resultado significativo (p < 0.05) y merece una investigación más profunda.

Seguimiento: Qué pasaría si en lugar de preguntar 
* *¿Cuál es la probabilidad de que más del Candidato A obtuviera más votos de los informados?*, 

preguntáramos 

* *¿Cuál es la probabilidad de que más del Candidato A obtuviera una cantidad de votos significativamente diferente a la informada?*

Es decir, ¿pasamos ahora del caso de una cola al caso de dos colas? Sigamos asumiendo que alguien ha preguntado si cumple con el criterio de $p = 0.05$

In [ ]:
xs = np.linspace(0, 1, 500)
ps = norm.pdf(xs, reported, sigma)

plt.plot(xs, ps, label="Proporcion incierta")
plt.axvline(sample, ls="--", label="Muestra")
limits = norm.ppf([0.025, 0.975], reported, sigma)
plt.fill_between(xs, ps, 0, alpha=0.2, where=(xs<limits[0])|(xs>limits[1]), label="Significanción")
plt.legend(loc=2)
plt.xlim(0.2, 0.65);

Entonces parece que no es $p<0.05$ significativo para el caso de dos colas. O más formalmente:

In [ ]:
# Usando z-scores
z_score = (sample - reported) / sigma
z_score_needed_for_significance = norm.ppf(0.975)
print(f"{z_score:.3f} es menos que {z_score_needed_for_significance:.3f}")

# Usando p-values
p_value_two_tailed = 2 * norm.sf(z_score)
print(f"{p_value_two_tailed:.3f} es > 0.05")